In [1]:
versioninfo()

Julia Version 1.0.5
Commit 3af96bcefc (2019-09-09 19:06 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i7-2620M CPU @ 2.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.0 (ORCJIT, sandybridge)


In [2]:
#import Pkg; Pkg.add("Arpack")
#import Pkg; Pkg.add("ArgParse")
#import Pkg; Pkg.add("LinearMaps")

using LinearAlgebra
using Arpack
using SparseArrays
using LinearMaps
#using ArgParse

In [3]:
function get_combination(NOS,NOD)
    combination = zeros(Int64,(NOS,NOD))
    for j in 1:NOD
        for i in 1:NOS
            combination[i,j] = f_combination(i,j)
        end
    end
    return combination
end

get_combination (generic function with 1 method)

In [4]:
function f_combination(n,k)
    if n<k
        return 0
    end
    nCk = 1
    for i in 1:k
        nCk = nCk * (n-k+i)
        nCk = div(nCk,i)
    end
    return nCk
end

f_combination (generic function with 1 method)

In [5]:
combination = get_combination(10,5)
#println(combination)
#show(combination)
display(combination)
println()

for n in 0:4
    for k in 0:n
        println(n," ",k," ",f_combination(n,k))
    end
    println()
end

10×5 Array{Int64,2}:
  1   0    0    0    0
  2   1    0    0    0
  3   3    1    0    0
  4   6    4    1    0
  5  10   10    5    1
  6  15   20   15    6
  7  21   35   35   21
  8  28   56   70   56
  9  36   84  126  126
 10  45  120  210  252


0 0 1

1 0 1
1 1 1

2 0 1
2 1 2
2 2 1

3 0 1
3 1 3
3 2 3
3 3 1

4 0 1
4 1 4
4 2 6
4 3 4
4 4 1



In [6]:
function insertion_sort!(a,NOD)
    for i in 2:NOD
        j = i - 1
        temp = a[i]
        while a[j] > temp
            a[j+1] = a[j]
            j = j - 1
            if j==0
                break
            end
        end
        a[j+1] = temp
    end
    return 0
end

insertion_sort! (generic function with 1 method)

In [7]:
a = [16:-1:1;]
#display(a)
println(a)
insertion_sort!(a,16)
#display(a)
println(a)

[16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


In [8]:
function inv_list(ni,NOD,combination)
    val_inv_list = ni[1]
    for i in 2:NOD
        val_inv_list = val_inv_list + combination[ni[i]-1,i]
    end
    return val_inv_list
end

inv_list (generic function with 1 method)

In [9]:
combination = get_combination(10,5)
ni = [1:10;]
display(combination)
println(ni)
for i in 1:5
    val_inv_list = inv_list(ni,i,combination)
    println(i," ",val_inv_list)
end

10×5 Array{Int64,2}:
  1   0    0    0    0
  2   1    0    0    0
  3   3    1    0    0
  4   6    4    1    0
  5  10   10    5    1
  6  15   20   15    6
  7  21   35   35   21
  8  28   56   70   56
  9  36   84  126  126
 10  45  120  210  252

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
1 1
2 1
3 1
4 1
5 1


In [10]:
function qsort_w_order!(a,o,first,last)
    x = a[div(first+last,2)]
    i = first
    j = last
    while true
        while a[i] < x
            i = i + 1
        end
        while x < a[j]
            j = j - 1
        end
        if i >= j
            break
        end
        t8 = a[i];  a[i] = a[j];  a[j] = t8
        t  = o[i];  o[i] = o[j];  o[j] = t
        i = i + 1
        j = j - 1
    end
    if first < i - 1
        qsort_w_order!(a,o,first,i-1)
    end
    if j + 1 < last
        qsort_w_order!(a,o,j+1,last)
    end
    return 0
end

qsort_w_order! (generic function with 1 method)

In [11]:
a = [16:-1:1;]
o = [32:-1:17;]
println(a)
println(o)
qsort_w_order!(a,o,4,12)
println(a)
println(o)

[16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
[32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17]
[16, 15, 14, 5, 6, 7, 8, 9, 10, 11, 12, 13, 4, 3, 2, 1]
[32, 31, 30, 21, 22, 23, 24, 25, 26, 27, 28, 29, 20, 19, 18, 17]


In [12]:
function list_fly(t,NOD,NOS,combination)
    ni = zeros(Int64,NOD)
    s = t
    j = NOS - 1
    for i in NOD:-1:2
        b, j0 = binary_search(s,combination[:,i],i,j)
        j = j0 - 1
        ni[i] = j0
        s = s - combination[j,i]
    end
    ni[1] = s
    return ni
end

list_fly (generic function with 1 method)

In [13]:
function binary_search(s,list_s,ls,le)
    bmin = ls; bmax = le
    while true
        b = bmin + div(bmax-bmin,2)
        if s < list_s[b]
            bmax = b - 1
        elseif list_s[b] < s
            bmin = b + 1
        else
            bmin = b
            return b, bmin
        end
        if bmin > bmax
            b = -1
            return b, bmin
        end
    end
    return b, bmin
end

binary_search (generic function with 1 method)

In [14]:
function list_to_state_no_duplication(st_list,NOS)
    string01 = ""
    for i in 1:NOS
        if i in st_list
            string01 = string01 * "1" # down
        else
            string01 = string01 * "0" # up
        end
    end
    return string01
end

list_to_state_no_duplication (generic function with 1 method)

In [15]:
function list_to_state(st_list,NOS)
    list01 = zeros(Int64,NOS)
    for i in st_list
        list01[i] += 1
    end
    list01 = mod.(list01,2)
    string01 = join(list01)
    return string01
end

list_to_state (generic function with 1 method)

In [16]:
NOS = 6 # number of sites
NOD = 3 # number of down spins
combination = get_combination(NOS,NOD)
THS = combination[NOS,NOD] # total Hilbert space
display(combination)
println(THS)
lv = THS

for i in 1:lv
    st_list = list_fly(i,NOD,NOS,combination)
    st_01 = list_to_state(st_list,NOS)
    st_idx = inv_list(st_list,NOD,combination)
    println(i," ",st_list," ",st_01," ",st_idx)
end

6×3 Array{Int64,2}:
 1   0   0
 2   1   0
 3   3   1
 4   6   4
 5  10  10
 6  15  20

20
1 [1, 2, 3] 111000 1
2 [1, 2, 4] 110100 2
3 [1, 3, 4] 101100 3
4 [2, 3, 4] 011100 4
5 [1, 2, 5] 110010 5
6 [1, 3, 5] 101010 6
7 [2, 3, 5] 011010 7
8 [1, 4, 5] 100110 8
9 [2, 4, 5] 010110 9
10 [3, 4, 5] 001110 10
11 [1, 2, 6] 110001 11
12 [1, 3, 6] 101001 12
13 [2, 3, 6] 011001 13
14 [1, 4, 6] 100101 14
15 [2, 4, 6] 010101 15
16 [3, 4, 6] 001101 16
17 [1, 5, 6] 100011 17
18 [2, 5, 6] 010011 18
19 [3, 5, 6] 001011 19
20 [4, 5, 6] 000111 20


In [17]:
function j_flip_ni(i,j,n,NOD)
    nd = ones(Int64,NOD)
    kr = NOD
    for _kr in NOD:-1:1
        if j < n[_kr]
            kr = _kr
            continue
        elseif j > n[_kr]
            kr = _kr
            break
        else
            fill!(nd,0)
            kr = _kr
            break
        end
    end
    if nd[NOD] == 1 # S+_i S-_j
        kl = 1
        for _kl in 1:kr
            if i == n[_kl]
                kl = _kl
                break
            end
            kl = _kl+1
        end
        nd[kl:kr-1] = n[kl+1:kr]
        nd[kr] = j
    else # S-_i S+_j
        kl = 1
        for _kl in 1:kr
            if i < n[_kl]
                kl = _kl
                break
            end
            kl = _kl+1
        end
        nd[kl] = i
        nd[kl+1:kr] = n[kl:kr-1]
    end
    nd[1:kl-1] = n[1:kl-1]
    nd[kr+1:NOD] = n[kr+1:NOD]
    return nd
end

j_flip_ni (generic function with 1 method)

In [18]:
NOS = 4 # number of sites
NOD = 2 # number of down spins
combination = get_combination(NOS,NOD)
THS = combination[NOS,NOD] # total Hilbert space
display(combination)
println(THS)
println()
for sp in 1:THS
    st_list = list_fly(sp,NOD,NOS,combination)
    println(sp," ",st_list," ",list_to_state(st_list,NOS))
#    st_idx = inv_list(st_list,NOD,combination)
#    println(sp," ",st_list," ",list_to_state(st_list,NOS)," ",st_idx)
    for i in 1:NOS
        for j in i+1:NOS
            nd = j_flip_ni(i,j,st_list,NOD)
            println(i," ",j," ",nd," ",list_to_state(nd,NOS))
#            nd_idx = inv_list(nd,NOD,combination)
#            println(i," ",j," ",nd," ",list_to_state(nd,NOS)," ",nd_idx)
        end
    end
    println()
end

4×2 Array{Int64,2}:
 1  0
 2  1
 3  3
 4  6

6

1 [1, 2] 1100
1 2 [1, 1] 0000
1 3 [2, 3] 0110
1 4 [2, 4] 0101
2 3 [1, 3] 1010
2 4 [1, 4] 1001
3 4 [1, 2] 1100

2 [1, 3] 1010
1 2 [2, 3] 0110
1 3 [1, 1] 0000
1 4 [3, 4] 0011
2 3 [1, 2] 1100
2 4 [1, 3] 1010
3 4 [1, 4] 1001

3 [2, 3] 0110
1 2 [1, 3] 1010
1 3 [1, 2] 1100
1 4 [2, 3] 0110
2 3 [2, 2] 0000
2 4 [3, 4] 0011
3 4 [2, 4] 0101

4 [1, 4] 1001
1 2 [2, 4] 0101
1 3 [3, 4] 0011
1 4 [1, 1] 0000
2 3 [1, 4] 1001
2 4 [1, 2] 1100
3 4 [1, 3] 1010

5 [2, 4] 0101
1 2 [1, 4] 1001
1 3 [2, 4] 0101
1 4 [1, 2] 1100
2 3 [3, 4] 0011
2 4 [2, 2] 0000
3 4 [2, 3] 0110

6 [3, 4] 0011
1 2 [3, 4] 0011
1 3 [1, 4] 1001
1 4 [1, 3] 1010
2 3 [2, 4] 0101
2 4 [2, 3] 0110
3 4 [3, 3] 0000



In [19]:
for f1 in [true,false]
    for f2 in [true,false]
        f3 = xor(f1,f2)
        println(f1," ",f2," ",f3)
    end
end

true true false
true false true
false true true
false false false


In [20]:
function make_full_hamiltonian(lv,combination,NOD,NOxxz,p_xxz,sJint,NOS)
    Ham = zeros(Float64,(lv,lv))
    for i in 1:lv
        st_list = list_fly(i,NOD,NOS,combination)
        for j in 1:NOxxz
            f1 = p_xxz[1,j] in st_list
            f2 = p_xxz[2,j] in st_list
            if xor(f1,f2)
                Ham[i,i] = Ham[i,i] - sJint[j,2]
                ni = j_flip_ni(p_xxz[1,j],p_xxz[2,j],st_list,NOD)
                id = inv_list(ni,NOD,combination)
                Ham[i,id] = Ham[i,id] + sJint[j,1]
            else
                Ham[i,i] = Ham[i,i] + sJint[j,2]
            end
        end
    end
    return Ham
end

make_full_hamiltonian (generic function with 1 method)

In [21]:
NOS = 4 # number of sites
NOD = 2 # number of down spins
NOxxz = NOS # number of XXZ interaction
combination = get_combination(NOS,NOD)
THS = combination[NOS,NOD] # total Hilbert space
display(combination)
println(THS)

p_xxz = zeros(Int64,(2,NOxxz))
Jint = zeros(Float64,(NOxxz,2)) # Jint[NOxxz,1] --> Jint_x, Jint[NOxxz,2] --> Jint_z
sJint = zeros(Float64,(NOxxz,2)) # sJint[NOxxz,1] --> sJint_x, sJint[NOxxz,2] --> sJint_z
for i in 1:NOS
    p_xxz[1,i] = mod(i-1,NOS)+1
    p_xxz[2,i] = mod(i,NOS)+1
    if p_xxz[1,i] > p_xxz[2,i] # assume i<j for pair (i,j)
        tmp = p_xxz[1,i]
        p_xxz[1,i] = p_xxz[2,i]
        p_xxz[2,i] = tmp
    end
end
fill!(Jint,1.0)
sJint[:,1] = 0.5 .* Jint[:,1]
sJint[:,2] = 0.25 .* Jint[:,2]
println(p_xxz)
println(Jint)
println(sJint)
println()

Ham = make_full_hamiltonian(THS,combination,NOD,NOxxz,p_xxz,sJint,NOS)
display(Ham)
for i in 1:THS
    for j in 1:THS
        if abs(Ham[i,j]) > 1e-6
            println(i," ",j," ",Ham[i,j])
        end
    end
end
println()

4×2 Array{Int64,2}:
 1  0
 2  1
 3  3
 4  6

6
[1 2 3 1; 2 3 4 4]


6×6 Array{Float64,2}:
 0.0   0.5  0.0  0.0   0.5  0.0
 0.5  -1.0  0.5  0.5   0.0  0.5
 0.0   0.5  0.0  0.0   0.5  0.0
 0.0   0.5  0.0  0.0   0.5  0.0
 0.5   0.0  0.5  0.5  -1.0  0.5
 0.0   0.5  0.0  0.0   0.5  0.0

[1.0 1.0; 1.0 1.0; 1.0 1.0; 1.0 1.0]
[0.5 0.25; 0.5 0.25; 0.5 0.25; 0.5 0.25]

1 2 0.5
1 5 0.5
2 1 0.5
2 2 -1.0
2 3 0.5
2 4 0.5
2 6 0.5
3 2 0.5
3 5 0.5
4 2 0.5
4 5 0.5
5 1 0.5
5 3 0.5
5 4 0.5
5 5 -1.0
5 6 0.5
6 2 0.5
6 5 0.5



In [22]:
function ham_to_vec_wave_vector!(lv,combination,NOD,NOxxz,p_xxz,sJint,NOS)
    function get_vec!(v0::AbstractVector,v1::AbstractVector) ## v0: new output, v1: old input
        for i = 1:lv
            v0[i] = 0.0 + 0.0im
            st_list = list_fly(i,NOD,NOS,combination)
            for j in 1:NOxxz
                f1 = p_xxz[1,j] in st_list
                f2 = p_xxz[2,j] in st_list
                if xor(f1,f2)
                    v0[i] = v0[i] - sJint[j,2] * v1[i]
                    ni = j_flip_ni(p_xxz[1,j],p_xxz[2,j],st_list,NOD)
                    id = inv_list(ni,NOD,combination)
                    v0[i] = v0[i] + sJint[j,1] * v1[id]
                else
                    v0[i] = v0[i] + sJint[j,2] * v1[i]
                end
            end
        end
        return v0
    end
    (v0,v1) -> get_vec!(v0,v1)
end

ham_to_vec_wave_vector! (generic function with 1 method)

In [23]:
NOS = 4 # number of sites
NOD = 2 # number of down spins
NOxxz = NOS # number of XXZ interaction
combination = get_combination(NOS,NOD)
THS = combination[NOS,NOD] # total Hilbert space
display(combination)
println(THS)

p_xxz = zeros(Int64,(2,NOxxz))
Jint = zeros(Float64,(NOxxz,2)) # Jint[NOxxz,1] --> Jint_x, Jint[NOxxz,2] --> Jint_z
sJint = zeros(Float64,(NOxxz,2)) # sJint[NOxxz,1] --> sJint_x, sJint[NOxxz,2] --> sJint_z
for i in 1:NOS
    p_xxz[1,i] = mod(i-1,NOS)+1
    p_xxz[2,i] = mod(i,NOS)+1
    if p_xxz[1,i] > p_xxz[2,i] # assume i<j for pair (i,j)
        tmp = p_xxz[1,i]
        p_xxz[1,i] = p_xxz[2,i]
        p_xxz[2,i] = tmp
    end
end
fill!(Jint,1.0)
sJint[:,1] = 0.5 .* Jint[:,1]
sJint[:,2] = 0.25 .* Jint[:,2]
println(p_xxz)
println(Jint)
println(sJint)
println()

get_vec_LM = ham_to_vec_wave_vector!(THS,combination,NOD,NOxxz,p_xxz,sJint,NOS)
Ham = LinearMap(get_vec_LM,THS;ismutating=true,issymmetric=true)
println(Ham)
println()

ene,vec = eigs(Ham,nev=5,which=:SR)
println("# energies")
println(ene)
println()
println("# vectors")
for i in 1:length(ene)
    println(i,vec[:,i])
end

4×2 Array{Int64,2}:
 1  0
 2  1
 3  3
 4  6

6
[1 2 3 1; 2 3 4 4]
[1.0 1.0; 1.0 1.0; 1.0 1.0; 1.0 1.0]
[0.5 0.25; 0.5 0.25; 0.5 0.25; 0.5 0.25]

LinearMaps.FunctionMap{Float64}(getfield(Main, Symbol("##3#5")){getfield(Main, Symbol("#get_vec!#4")){Int64,Array{Int64,2},Int64,Int64,Array{Int64,2},Array{Float64,2},Int64}}(getfield(Main, Symbol("#get_vec!#4")){Int64,Array{Int64,2},Int64,Int64,Array{Int64,2},Array{Float64,2},Int64}(6, [1 0; 2 1; 3 3; 4 6], 2, 4, [1 2 3 1; 2 3 4 4], [0.5 0.25; 0.5 0.25; 0.5 0.25; 0.5 0.25], 4)), 6, 6; ismutating=true, issymmetric=true, ishermitian=true, isposdef=false)

# energies
[-2.0, -1.0, -1.11597e-16, -1.33402e-17, 1.89191e-18]

# vectors
1[0.288675, -0.57735, 0.288675, 0.288675, -0.57735, 0.288675]
2[0.0, -0.707107, 2.22045e-16, 1.11022e-16, 0.707107, 5.55112e-17]
3[0.793685, 1.11022e-16, -0.442207, -0.413173, -5.55112e-17, 0.0616944]
4[0.282697, -5.55112e-17, -0.15849, 0.603948, 0.0, -0.728155]
5[-0.200367, 5.55112e-17, -0.727554, 0.463179, 0.0, 0.464741]
